In [ ]:
import string
import random
import re
import pandas as pd

from simplemma import text_lemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV 

In [ ]:
df = pd.read_csv("magaza_yorumlari_duygu_analizi.csv", encoding="utf-16")
df.dropna(inplace=True)
df.head()

In [ ]:
df = df.loc[df["Durum"].isin(["Olumlu", "Olumsuz"])]
df["Durum"].value_counts()

In [ ]:
def etiketle(row):
    if row["Durum"] == "Olumlu":
        return 1
    return 0


df["label"] = df.apply(lambda row: etiketle(row), axis=1)

df.head()

In [ ]:
stop_words = {"a", "acaba", "ama", "ancak", "artık", "asla", "aslında", "az", "b", "bana", "bazen", "bazı", "bazıları", "bazısı", "belki", "ben", "beni", "benim", "beş", "bile", "bir", "birçoğu", "birçok", "birçokları", "biri", "birisi", "birkaç", "birkaçı", "birşey", "birşeyi", "biz", "bize", "bizi", "bizim", "böyle", "böylece", "bu", "buna", "bunda", "bundan", "bunu", "bunun", "burada", "bütün", "c", "ç", "çoğu", "çoğuna", "çoğunu", "çok", "çünkü", "d", "da", "daha", "de", "değil", "demek", "diğer", "diğeri", "diğerleri", "diye", "dokuz", "dolayı", "dört", "e", "elbette", "en", "f", "fakat", "falan", "felan", "filan", "g", "gene", "gibi", "ğ", "h", "hâlâ", "hangi", "hangisi", "hani", "hatta", "hem", "henüz", "hep", "hepsi", "hepsine", "hepsini", "her", "her biri", "herkes", "herkese", "herkesi", "hiç", "hiç kimse", "hiçbiri", "hiçbirine", "hiçbirini", "ı", "i", "için", "içinde", "iki", "ile", "ise", "işte",
              "j", "k", "kaç", "kadar", "kendi", "kendine", "kendini", "ki", "kim", "kime", "kimi", "kimin", "kimisi", "l", "m", "madem", "mı", "mı", "mi", "mu", "mu", "mü", "mü", "n", "nasıl", "ne", "ne kadar", "ne zaman", "neden", "nedir", "nerde", "nerede", "nereden", "nereye", "nesi", "neyse", "niçin", "niye", "o", "on", "ona", "ondan", "onlar", "onlara", "onlardan", "onların", "onların", "onu", "onun", "orada", "oysa", "oysaki", "ö", "öbürü", "ön", "önce", "ötürü", "öyle", "p", "r", "rağmen", "s", "sana", "sekiz", "sen", "senden", "seni", "senin", "siz", "sizden", "size", "sizi", "sizin", "son", "sonra", "ş", "şayet", "şey", "şeyden", "şeye", "şeyi", "şeyler", "şimdi", "şöyle", "şu", "şuna", "şunda", "şundan", "şunlar", "şunu", "şunun", "t", "tabi", "tamam", "tüm", "tümü", "u", "ü", "üç", "üzere", "v", "var", "ve", "veya", "veyahut", "y", "ya", "ya da", "yani", "yedi", "yerine", "yine", "yoksa", "z", "zaten"}


def clean_text(text):
    text = " ".join(text.split())
    text = text.lower()
    text = text.replace("\\n", " ")
    text = re.sub("[0-9]+", "", text)
    text = re.sub("%|(|)|-", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = " ".join([t for t in text.split() if t not in stop_words])
    return " ".join(text_lemmatizer(text, lang="tr"))


df["clean"] = df.apply(lambda row: clean_text(row["Görüş"]), axis=1)

In [ ]:
for _ in range(10):
    idx = random.randint(0, len(df)-1)
    print(df.iloc[idx]["clean"])
    print("-" * 20)

In [ ]:
df.head(10)

In [ ]:
X = df["clean"].to_numpy()
y = df["label"].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
clf_NB = MultinomialNB()
clf_NB.fit(X_train, y_train)

clf_NB.score(X_test, y_test)

In [ ]:
clf_DT = DecisionTreeClassifier()
clf_DT.fit(X_train, y_train)

clf_DT.score(X_test, y_test)

In [ ]:
clf_SVM = SVC(kernel="linear")
clf_SVM.fit(X_train, y_train)

clf_SVM.score(X_test, y_test)

In [ ]:
clf_SVM = SVC(kernel="rbf")
clf_SVM.fit(X_train, y_train)

clf_SVM.score(X_test, y_test)

In [ ]:
param_grid = [
  {"C": [1, 10, 100], "kernel": ["linear"]},
  {"C": [1, 10, 100], "gamma": [0.001, 0.0001], "kernel": ["rbf"]},
 ]
  
grid = GridSearchCV(SVC(), param_grid, verbose = 3) 
  
grid.fit(X_train, y_train) 

print(grid.best_params_) 

In [ ]:
grid.best_estimator_.score(X_test, y_test)